# Labor Force Statistics from CPS Microdata

Brian Dew, @bd_econ, brian.w.dew@gmail.com

In [1]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

dt = cps_date()
if dt > pd.to_datetime('2022-02-01'):
    pcdt = dt - pd.DateOffset(years=3)
else:
    pcdt = dt - pd.DateOffset(years=2)
def to_date(ym):
    return pd.to_datetime(f'{ym[0]}-{ym[1]}-01')

### LFS by AGE

In [ ]:
lfs2 = lambda x: np.where(((x.AGE < 16) | (x.NILFREASON == 'School')), 'Child/Student',
                 np.where(((x.LFS == 'Unemployed') | (x.NILFREASON == 'Discouraged') | 
                           (x.NILFREASON == 'Other')), 'Unemployed/Discouraged/Other',
                 np.where(x.LFS == 'Employed', 'Employed', x.NILFREASON)))

cols = ['YEAR', 'MONTH', 'AGE', 'NILFREASON', 'LFS', 'PWSSWGT']
df = (cps_1mo(cps_dir, dt, cols).assign(LFS2 = lfs2))
df2 = df.copy()
df2.loc[:,'AGE'] = df.AGE + 1
df3 = df.copy()
df3.loc[:,'AGE'] = df.AGE - 1
df = df.append(df2).append(df3)
data = (df.groupby(['AGE', 'LFS2']).PWSSWGT.sum().unstack()
          .divide(df.groupby('AGE').PWSSWGT.sum(), axis=0))
data.columns.name = ''
data = data.fillna(0) * 100
data.to_csv(data_dir / 'lfs_age.csv', index_label='AGE')

In [ ]:
df = (cps_1mo(cps_dir, pcdt, cols).assign(LFS2 = lfs2))
df2 = df.copy()
df2.loc[:,'AGE'] = df.AGE + 1
df3 = df.copy()
df3.loc[:,'AGE'] = df.AGE - 1
df = df.append(df2).append(df3)
data2 = (df.groupby(['AGE', 'LFS2']).PWSSWGT.sum().unstack()
           .divide(df.groupby('AGE').PWSSWGT.sum(), axis=0))
data2.columns.name = ''
data2 = data2.fillna(0) * 100
result = (data - data2).loc[15:].rename({'Child/Student': 'Student'}, axis=1)
result.to_csv(data_dir / 'lfs_age_ch.csv', index_label='AGE')

### Overview data and text

In [ ]:
yr_wt = {y: 'PWSSWGT' if y > 1993 else 'BASICWGT' 
         for y in range(1989, 2023)}

cols = ['LFS', 'AGE', 'MONTH', 'YEAR', 'FEMALE']

df = pd.concat([(pd.read_feather(cps_dir / f'cps{y}.ft', 
                                 columns=cols + [w])
                   .rename({w: 'WGT'}, axis=1)) 
                for y, w in yr_wt.items()], sort=False)
df['DATE'] = pd.to_datetime(dict(year=df.YEAR, 
                                 month=df.MONTH, day=1))
data = (df.groupby(['DATE', 'LFS']).WGT.sum()
          .unstack().rename({'nan': 'Children'}, axis=1)
        / 1_000_000)
data.to_csv(data_dir/ 'cps_lfs.csv', index_label='date')

In [ ]:
data = pd.read_csv(data_dir/ 'cps_lfs.csv', 
                   index_col='date', parse_dates=True)

datelt = dtxt(data.index[-1])['mon1']
dateltsh = dtxt(data.index[-1])['mon2']
dateyr = dtxt(data.index[-13])['mon1']
dateprepan = dtxt(data.index[-25])['mon1']
dateshprepan = dtxt(data.index[-25])['mon2']
dateyrsh = dtxt(data.index[-13])['mon2']
datepr = f"{dateyr} to {datelt}"
datepr2 = f"{dateprepan} to {datelt}"

write_txt(text_dir / 'cps_lfs_date.txt', datelt)
write_txt(text_dir / 'cps_lfs_datesh.txt', dateltsh)
write_txt(text_dir / 'cps_lfs_dateyr.txt', dateyr)
write_txt(text_dir / 'cps_lfs_datepc.txt', dtxt(pcdt)['mon1'])
write_txt(text_dir / 'cps_lfs_dateprepan.txt', dateprepan)
write_txt(text_dir / 'cps_lfs_dateshprepan.txt', dateshprepan)
write_txt(text_dir / 'cps_lfs_dateyrsh.txt', dateyrsh)
write_txt(text_dir / 'cps_lfs_datepr.txt', datepr)
write_txt(text_dir / 'cps_lfs_date2pr.txt', datepr2)

epop = data["Employed"].iloc[-1] / data.iloc[-1].sum() * 100

In [ ]:
text = (f'As of {datelt}, {data["Employed"].iloc[-1]:.1f} '+
        'million people are employed (including self-'+
        'employment). ')
print(text, '\n')
write_txt(text_dir / 'cps_lfs0.txt', text)

text = ('The number of people who are employed divided by '+
        'the total population is the employment rate or '+
        'employment-to-population ratio, which is '+
        f'{epop:.1f} percent as of {datelt}.')
print(text, '\n')
write_txt(text_dir / 'cps_lfs.txt', text)

unemp = (data["Unemployed"].iloc[-1] / 
         (data["Unemployed"].iloc[-1] + 
          data["Employed"].iloc[-1])) * 100
lfpr = ((data["Unemployed"].iloc[-1] + 
         data["Employed"].iloc[-1]) / 
         data.iloc[-1].sum()) * 100

unemptot = data["Unemployed"].iloc[-1]
text = (f'As of {datelt}, there are {unemptot:.1f} million '+
        'unemployed people. The combined group of employed '+
        'and unemployed people is the labor force. The number '+
        'of unemployed people divided by the number of people '+
        'in the labor force is the unemployment rate, '+
        f'currently {unemp:.1f} percent. The number of people '+
        'in the labor force divided by the total population '+
        'is the labor force participation rate, currently '+
        f'{lfpr:.1f} percent.')
print(text, '\n')
write_txt(text_dir / 'cps_lfs2.txt', text)
    
cols2 = ['LFS', 'AGE', 'MONTH', 'YEAR', 'FEMALE', 
         'NILFREASON', 'PWSSWGT']

v = (cps_1mo(cps_dir, cps_date(), cols2)
     .groupby('NILFREASON').PWSSWGT.sum()
     .divide(1_000_000)
     .apply('({:.1f} million)'.format)
     .str.replace('.0', '', regex=False))

kids = data["Children"].iloc[-1]
nilfpop = data[['NILF', 'Children']].iloc[-1].sum()
text = ('Nonparticipants usually comprise about half '+
        f'of the population, and total {nilfpop:.1f} million '+
        f'in {datelt}. The category includes children '+
        f'({kids:.1f} million), students {v["School"]}, '+
        f'unpaid caregivers {v["Family"]}, those unable to '+
        f'work due to disability or illness {v["Disabled/Ill"]}, '+
        f'those who want a job but have given up looking '+
        f'{v["Discouraged"]}, and retirees and the elderly '+
        f'{v["Retired"]}.')
write_txt(text_dir / 'cps_lfs3.txt', text)
    
data['Total'] = (data[['Employed', 'Unemployed', 'NILF']]
                 .sum(axis=1))
result = data.drop('Children', axis=1).resample('QS').mean()

gc = growth_contrib_ann(result, 'Total').dropna()
gc.to_csv(data_dir / 'cps_lfs2.csv', index_label='date')
print(text)

### LFS Table

In [ ]:
cols = ['LFS', 'BASICWGT', 'AGE', 'MONTH', 'YEAR', 'MJH',
        'FEMALE', 'NILFREASON', 'UNEMPTYPE', 'PTECON', 'WORKFT']

df = cps_1mo(cps_dir, cps_date(), cols).query('AGE > 15')
df2 = cps_1mo(cps_dir, cps_date() - pd.DateOffset(years=2), cols).query('AGE > 15')

age_groups = [(16, 30), (30, 60), (60, 86)]
sex = [0, 1]
d = {'Population': {'query': 'BASICWGT > 0'}, 
     '\hspace{2mm}Employed': {'query': 'LFS == "Employed"'}, 
     '\hspace{4mm}Multiple jobs': {'query': 'MJH == 1'},     
     '\hspace{4mm}Full-time': {'query': 'WORKFT == 1'},  
     '\hspace{4mm}Part-time': {'query': 'WORKFT == 0'}, 
     '\hspace{6mm}Economic reasons': {'query': 'PTECON == 1'},
     '\hspace{2mm}Unemployed': {'query': 'LFS == "Unemployed"'}, 
     '\hspace{2mm}Not in Labor Force': {'query': 'LFS == "NILF"'}, 
     '\hspace{4mm}Discouraged': {'query': 'NILFREASON == "Discouraged"'}, 
     '\hspace{4mm}Disabled/Ill': {'query': 'NILFREASON == "Disabled/Ill"'}, 
     '\hspace{4mm}Family/Care': {'query': 'NILFREASON == "Family"'}, 
     '\hspace{4mm}School': {'query': 'NILFREASON == "School"'}, 
     '\hspace{4mm}Retirement': {'query': 'NILFREASON == "Retired"'}}

d2 = {k: {} for k, v in d.items()}

lf_groups = list(zip(d.keys(), [d[i]['query'] for i in d.keys()])) 

for name, query in lf_groups:
    totval = df.query(query).BASICWGT.sum()/1000
    d[name]['Total, 16+'] = f'{totval:,.0f}'
    chval = totval - df2.query(query).BASICWGT.sum()/1000
    d2[name]['Total, 16+'] = f'{chval:,.0f}'

for sex, (agemin, agemax) in itertools.product(sex, age_groups):
    data = df.query('@agemin <= AGE < @agemax and FEMALE == @sex')
    data2 = df2.query('@agemin <= AGE < @agemax and FEMALE == @sex')
    group = f'{["Women" if sex==1 else "Men"][0]}, {agemin}--{agemax-1}'.replace('--85', '+')
    for name, query in lf_groups:
        tmp = data.query(query).BASICWGT.sum()/1000
        tmp2 = data2.query(query).BASICWGT.sum()/1000
        d[name][group] = f'{tmp:,.0f}'
        d2[name][group] = f'{tmp - tmp2:,.0f}'
        
(pd.DataFrame(d).T.drop('query', axis=1).to_csv(data_dir / 'lfs_table1.tex', 
    sep='&', line_terminator='\\\ ', quotechar=' '))
pd.DataFrame(d2).T.to_csv(data_dir / 'lfs_table2.tex', sep='&', line_terminator='\\\ ', quotechar=' ')

In [ ]:
# One year change
cols = ['LFS', 'BASICWGT', 'AGE', 'MONTH', 'YEAR', 'MJH',
        'FEMALE', 'NILFREASON', 'UNEMPTYPE', 'PTECON', 'WORKFT']

df = cps_1mo(cps_dir, cps_date(), cols).query('AGE > 15')
df2 = cps_1mo(cps_dir, cps_date() - pd.DateOffset(years=1), cols).query('AGE > 15')

age_groups = [(16, 30), (30, 60), (60, 86)]
sex = [0, 1]
d = {'Population': {'query': 'BASICWGT > 0'}, 
     '\hspace{2mm}Employed': {'query': 'LFS == "Employed"'}, 
     '\hspace{4mm}Multiple jobs': {'query': 'MJH == 1'},     
     '\hspace{4mm}Full-time': {'query': 'WORKFT == 1'},  
     '\hspace{4mm}Part-time': {'query': 'WORKFT == 0'}, 
     '\hspace{6mm}Economic reasons': {'query': 'PTECON == 1'},
     '\hspace{2mm}Unemployed': {'query': 'LFS == "Unemployed"'}, 
     '\hspace{2mm}Not in Labor Force': {'query': 'LFS == "NILF"'}, 
     '\hspace{4mm}Discouraged': {'query': 'NILFREASON == "Discouraged"'}, 
     '\hspace{4mm}Disabled/Ill': {'query': 'NILFREASON == "Disabled/Ill"'}, 
     '\hspace{4mm}Family/Care': {'query': 'NILFREASON == "Family"'}, 
     '\hspace{4mm}School': {'query': 'NILFREASON == "School"'}, 
     '\hspace{4mm}Retirement': {'query': 'NILFREASON == "Retired"'}}

d2 = {k: {} for k, v in d.items()}

lf_groups = list(zip(d.keys(), [d[i]['query'] for i in d.keys()])) 

for name, query in lf_groups:
    totval = df.query(query).BASICWGT.sum()/1000
    d[name]['Total, 16+'] = f'{totval:,.0f}'
    chval = totval - df2.query(query).BASICWGT.sum()/1000
    d2[name]['Total, 16+'] = f'{chval:,.0f}'

for sex, (agemin, agemax) in itertools.product(sex, age_groups):
    data = df.query('@agemin <= AGE < @agemax and FEMALE == @sex')
    data2 = df2.query('@agemin <= AGE < @agemax and FEMALE == @sex')
    group = f'{["Women" if sex==1 else "Men"][0]}, {agemin}--{agemax-1}'.replace('--85', '+')
    for name, query in lf_groups:
        tmp = data.query(query).BASICWGT.sum()/1000
        tmp2 = data2.query(query).BASICWGT.sum()/1000
        d[name][group] = f'{tmp:,.0f}'
        d2[name][group] = f'{tmp - tmp2:,.0f}'
        
print(datepr)
pd.DataFrame(d2).T        

### Employment rates by age, gender, education

In [ ]:
cols = ['MONTH', 'YEAR', 'LFS', 'PWSSWGT', 'AGE', 'EDUC', 'FEMALE']
df1 = cps_3mo(cps_dir, cps_date() - pd.DateOffset(months=24), cols)
df2 = cps_3mo(cps_dir, cps_date(), cols)

ages = [(16, 24), (25, 34), (35, 44), (45, 54), (55, 64), (65, 74)]
educ = [['LTHS', 'HS'], ['SC'], ['COLL', 'ADV'], ['LTHS', 'HS', 'SC', 'COLL', 'ADV']]
sex = [0, 1]

results = pd.DataFrame()
for a, e, s in itertools.product(ages, educ, sex):
    row_name = f'{a[0]}-{a[1]}'
    col_name = f'{" or ".join(e)} {["Female" if s == 1 else "Male"][0]}'
    for yr, df in [('Year Ago', df1), ('Latest', df2)]:
        data = df.query('@a[0] <= AGE <= @a[1] and EDUC in @e and FEMALE == @s')
        i = data.groupby('LFS').PWSSWGT.sum()
        results.loc[row_name, f'{yr} {col_name}'] = (i.Employed / i.sum()) * 100
        
results.iloc[1:, :-4].to_csv(data_dir / 'empgroups.csv', index_label='name')
results.iloc[:, -4:].to_csv(data_dir / 'empgroups2.csv', index_label='name')

### Union Membership

In [ ]:
union_membership_rate = lambda x: np.average(x['UNIONMEM'], weights=x['PWORWGT'])
union_coverage_rate = lambda x: np.average(x['UNION'], weights=x['PWORWGT'])

unmem, uncov = pd.Series(dtype='float64'), pd.Series(dtype='float64')

untot, nuntot = pd.Series(dtype='float64'), pd.Series(dtype='float64')

columns = ['MONTH', 'YEAR', 'LFS', 'PWORWGT', 'UNION', 'UNIONMEM']
for year in range(1989, 2023):
    df = (pd.read_feather(cps_dir / f'cps{year}.ft', columns=columns)
        .query('LFS == "Employed" and UNION in [0, 1]'))
    data1 = df.groupby(['YEAR', 'MONTH']).apply(union_membership_rate)
    data1.index = [pd.to_datetime(f'{ti[0]}-{ti[1]}-01') for ti in data1.index]
    unmem = unmem.append(data1)
    data2 = df.groupby(['YEAR', 'MONTH']).apply(union_coverage_rate)
    data2.index = [pd.to_datetime(f'{ti[0]}-{ti[1]}-01') for ti in data2.index]
    uncov = uncov.append(data2)
    df2 = df.query('UNIONMEM == 1')   
    data3 = df2.groupby(['YEAR', 'MONTH']).PWORWGT.sum()
    data3.index = [pd.to_datetime(f'{ti[0]}-{ti[1]}-01') for ti in data3.index]
    untot = untot.append(data3)
    df3 = df.query('UNIONMEM == 0')
    data4 = df3.groupby(['YEAR', 'MONTH']).PWORWGT.sum()
    data4.index = [pd.to_datetime(f'{ti[0]}-{ti[1]}-01') for ti in data4.index]
    nuntot = nuntot.append(data4)    
    
data = pd.DataFrame({'Membership': unmem, 'Coverage': uncov})
levels = pd.DataFrame({'Union': untot, 'Nonunion': nuntot})
data['Diff'] = data['Coverage'] - data['Membership']
final = (data.rolling(12).mean().dropna() * 100)
final.to_csv(data_dir / 'union.csv', index_label='date', float_format='%g')
final2 = (levels.rolling(12).mean().dropna()) / 1000000

ltdate = final.index[-1].strftime('%B %Y')
prevdate = final.index[-13].strftime('%B %Y')
prev2date = final.index[-25].strftime('%B %Y')

ltval = final['Membership'].iloc[-1]
prevval = final['Membership'].iloc[-13]
prev2val = final['Membership'].iloc[-25]

totvallt = final2['Union'].iloc[-1]
totnvallt = final2['Nonunion'].iloc[-1]

chlt = final2['Union'].diff(12).iloc[-1] * 1000000
chpr = final2['Union'].diff(12).iloc[-13] * 1000000

chnlt = final2['Nonunion'].diff(12).iloc[-1] * 1000000
chnpr = final2['Nonunion'].diff(12).iloc[-13] * 1000000

if chlt > 10:
    chlt_txt = f'increased by {round(chlt, -3):,.0f}'
elif chlt <= -10:
    chlt_txt = f'decreased by {abs(round(chlt, -3)):,.0f}'
else:
    chlt_txt = 'were virtually unchanged'
    
if chnlt > 10:
    chnlt_txt = f'increased by {round(chnlt, -3):,.0f}'
elif chnlt <= -10:
    chnlt_txt = f'decreased by {abs(round(chnlt, -3)):,.0f}'
else:
    chnlt_txt = 'were virtually unchanged'
    
text = (f'Over the 12 months ending {ltdate}, the share of jobs held '+
        f'by union and employee association members averaged {ltval:.1f} percent. '+
        f'In levels, there were {totvallt:.1f} million union jobs, and '+
        f'{totnvallt:.1f} million nonunion jobs, on average over the period. '+
        f'This union membership rate averaged {prevval:.1f} percent during the 12 '+
        f'months ending {prevdate}, and {prev2val:.1f} percent during the 12 '+
        f'months ending {prev2date}. Union jobs {chlt_txt} '+
        f'from {prevdate} to {ltdate}, while nonunion jobs {chnlt_txt}.')
write_txt(text_dir / 'union.txt', text)
print(text)

In [ ]:
union_membership_rate = lambda x: np.average(x['UNIONMEM'], weights=x['PWORWGT']) * 100
columns = ['MONTH', 'YEAR', 'LFS', 'PWORWGT', 'UNIONMEM', 'INDGRP']

df = pd.concat([pd.read_feather(cps_dir / f'cps{year}.ft', columns=columns)
                  .query('PWORWGT > 0 and UNIONMEM == UNIONMEM')
      for year in range(1989, 2023)])

df.INDGRP.cat = df.INDGRP.cat.remove_unused_categories()

data = df.groupby(['YEAR', 'MONTH', 'INDGRP']).apply(union_membership_rate).unstack()
data.index = [pd.to_datetime(f'{ti[0]}-{ti[1]}-01') for ti in data.index]
data.index.name = 'date'
data.columns.name = None
prev12 = data.iloc[-12:].mean()
lval = data.iloc[-1]
minval = data.min()
maxval = data.max()
width = maxval - minval
final = pd.concat([lval, prev12, minval, maxval, width], axis=1)
final.columns = ['latest', 'prev12', 'min', 'max', 'width']
final.index.name = 'name'
final = final[['min', 'width', 'latest']].sort_values('latest', ascending=False)
final['y'] = [0.13, -0.87, -1.87, -2.87, -3.87, -4.87, -5.87]
final.to_csv(data_dir / 'union_ind.csv', sep=';')

dec = (maxval - lval).sort_values()
dec1 = dec.index[-1]
decval1 = dec[-1]
decmaxdt = dtxt(data[dec1].idxmax())['mon1']
decmaxval = data[dec1].max()
decgrp = dec1.lower()
lowgroupmaxdt = dtxt(data[final.index[-1]].idxmax())['mon1']
lowgrp = final.index[-1].lower()


text = (f'{final.index[0]} has the highest union membership rate, '+
        f'at {final.latest.iloc[0]:.1f} percent as of {ltdate}, followed by '+
        f'{final.index[1].lower()} with {final.latest.iloc[1]:.1f} percent, '+
        f'and {final.index[2].lower()} with {final.latest.iloc[2]:.1f} percent. '+
        f'The {decgrp} industry '+
        'experienced the largest overall percentage point decrease '+
        'in union membership rates over the past 30 years, and is '+
        f'currently {decval1:.1f} percentage points below its {decmaxdt} '+
        f'rate of {decmaxval:.1f} percent. ')
txt2 = ('The lowest union membership rate '+
        f'is in {lowgrp} ({final.latest.iloc[-1]:.1f} percent). '+
        f'The union membership rate of the industry was {data[final.index[-1]].max():.1f} '+
        f'percent at its 30-year peak in {lowgroupmaxdt}. ')

mfglt = data['Manufacturing'].iloc[-1]
mfgpr = data['Manufacturing'].iloc[-13]
prdt = dtxt(data.index[-13])['mon1']
mfgpr2 = data['Manufacturing'].iloc[-25]
prdt2 = dtxt(data.index[-25])['mon1']

txt3 = (f'The manufacturing industry union membership rate was {mfglt:.1f} percent in {ltdate}, '+
        f'{mfgpr:.1f} percent in {prdt}, and {mfgpr2:.1f} percent in {prdt2}.')

if lowgrp == decgrp:
    text = text + txt3
else:
    text = text + txt2

write_txt(text_dir / 'union_ind.txt', text)
print(text)

### Matching YY -- Disability to Work Flows

In [ ]:
# Calculate monthly rate and store as raw file
cols = ['CPSID', 'AGE', 'FEMALE', 'WBHAO', 'NILFREASON', 
        'LFS', 'MIS', 'BASICWGT', 'LINENO']
emp = lambda x: np.where(x.LFS_y == 'Employed', 1, 0)
dates = pd.date_range('1997-01-01', cps_date(), freq='MS')
d = {}
for dt in dates:
    df1 = (cps_1mo(cps_dir, dt - pd.DateOffset(years=1), cols)
           .query('NILFREASON == "Disabled/Ill"'))
    df2 = cps_1mo(cps_dir, dt, cols)
    data = (pd.merge(df1, df2, on=['CPSID', 'LINENO', 'FEMALE', 
                                   'WBHAO'])
              .query('25 <= AGE_y <= 54'))
    data = ((data.loc[(data.AGE_x <= data.AGE_y) & 
                      (data.AGE_y - 2 <= data.AGE_x)])
                 .assign(EMP = emp))
    d[dt] = np.average(data.EMP, weights=data.BASICWGT_y) * 100
s = pd.Series(d).rename('Share')
s.to_csv(data_dir / 'disflow_raw.csv', index_label='date', 
         header=True)

In [ ]:
s = pd.read_csv(data_dir / 'disflow_raw.csv', 
                index_col='date', parse_dates=True)
df = s.rolling(12).mean().dropna()['Share']
df.to_csv(data_dir / 'disflow.csv', index_label='date', 
          header=True)
color = 'blue'
node = end_node(df, color, date='m', percent=True, 
                offset=True, size=1.2)
write_txt(text_dir/ 'disflow_node.txt', node)

latest = df.iloc[-1]
valavg = df.loc['2010-12-01':'2013-12-01'].mean()
val19 = df.loc['2019'].mean()
ltdate = dtxt(df.index[-1])['mon1']

text = (f'Over the year ending {ltdate}, {latest:.1f} '+
        'percent of 25 to 54 year olds who were out of '+
        'the labor force due to disability or illness '+
        f'one year prior became employed {c_line(color)}. '+
        f'Pre-pandemic, in 2019, {val19:.1f} percent of '+
        'those in the category found a job. '+
        'The one-year rate of job-finding has increased '+
        'substantially from its 2010--2013 average of '+
        f'{valavg:.1f} percent.')
write_txt(text_dir / 'disflow.txt', text)
print(text)

### Job switching rate

In [ ]:
cols = ['YEAR', 'MONTH', 'SAMEEMP', 'BASICWGT', 'LFS']
df = pd.concat([pd.read_feather(cps_dir / f'cps{year}.ft', columns=cols)
                  .query('LFS == "Employed" and SAMEEMP in [0, 1]') 
                for year in range(1994, 2023)])
newemp = (lambda x: x.query('SAMEEMP == 0').BASICWGT.sum()
                 / x.BASICWGT.sum())

data = (df.groupby(['YEAR', 'MONTH']).apply(newemp)).reset_index()
data['date'] = pd.to_datetime(dict(year=data.YEAR, month=data.MONTH, day=1))
data = data.set_index('date').drop(['YEAR', 'MONTH'], axis=1) * 100
sm = x13_arima_analysis(data)
res = pd.concat([sm.seasadj.rename('Monthly'), 
           sm.seasadj.rolling(3).mean().rename('MA3')], axis=1)
res.to_csv(data_dir / 'jobswitch.csv', index_label='date')

In [ ]:
res = pd.read_csv(data_dir / 'jobswitch.csv', index_col='date', 
                  parse_dates=True)
qcol = 'blue!80!black'
mcol = 'green!90!blue'
node = end_node(res['Monthly'], mcol, date='m', percent=True)
write_txt(text_dir/ 'jobswitch_node.txt', node)
ltdate = dtxt(res.index[-1])['mon1']
ltval = f'{res["Monthly"].iloc[-1]:.1f} percent'
ltqval = f'{res["MA3"].iloc[-1]:.1f} percent'
also = 'also ' if ltval == ltqval else ''
ltq = f'the three months ending {ltdate}'
pcdate = '2019-12-01'
pcqval = f'{res["MA3"].loc[pcdate]:.1f} percent'
text = ('More recent data show a slight increase in job switching rates. '+
        f'In {ltdate}, {ltval} of the workforce had a '+
        'different employer than the previous month, after seasonal '+
        f'adjustment {c_line(mcol)}. Smoothed data {also}show an average '+
        f'of {ltqval} of the workforce with a new employer during {ltq} '+
        f'{c_line(qcol)}. Prior to COVID-19, in {dtxt(pcdate)["qtr1"]}, '+
        f'a monthly average of {pcqval} of the workforce switched jobs.')
write_txt(text_dir / 'jobswitch.txt', text)
print(text)

### Young adults school/NILF

In [ ]:
sch_stat = lambda x: np.where(x.NILFREASON == 'School', 'School',
                     np.where(x.LFS == 'NILF', 'Other', 'LF'))
age_grp = lambda x: np.where(x.AGE < 22, '19-21', '22-24')
date = lambda x: pd.to_datetime(dict(year=x.YEAR, month=x.MONTH, day=1))

datecol = ['YEAR', 'MONTH']
cols = ['AGE', 'NILFREASON', 'BASICWGT', 'LFS'] + datecol

df = pd.concat([pd.read_feather(cps_dir / f'cps{year}.ft', columns=cols)
                  .query('18 < AGE < 25').assign(SCH_STAT = sch_stat, 
                                                 AGE_GRP = age_grp,
                                                 DATE = date) 
                for year in range(1994, 2023)])

res = pd.DataFrame()
for ag in ['19-21', '22-24']:
    td = df.query('AGE_GRP == @ag')
    data = ((td.groupby(['DATE', 'SCH_STAT']).BASICWGT.sum() / 
             td.groupby('DATE').BASICWGT.sum()) * 100).unstack()
    data.columns = [i + '_' + ag for i in data.columns]
    data.columns.name = ''
    data['Total_' + ag] = 100 - data['LF_' + ag]
    data = data.drop('LF_' + ag, axis=1)
    res = res.join(data, how='right')
    
ma = res.rolling(12).mean().dropna()
ma.to_csv(data_dir/ 'young_school.csv', index_label='date')

colors = [('School_', 'blue!50!cyan'),
          ('Other_', 'violet!85!black')]

for ag in ['19-21', '22-24']:
    node_dt = None if ag == '22-24' else 'm'
    nodeadj = 0 if ag == '22-24' else -0.25
    node = end_node(ma['Total_' + ag], 'orange!90!yellow', percent=True,
                    date=node_dt, offset=nodeadj, full_year=True)
    nodes = '\n'.join([node] + [end_node(ma[name + ag], color, percent=True) 
                       for name, color in colors])
    write_txt(text_dir/ f'young_nilf{ag}_node.txt', nodes)
    
t0019 = f'{ma.loc["2000-12-01", "Total_19-21"]:.1f}'
t0022 = f'{ma.loc["2000-12-01", "Total_22-24"]:.1f}'
t1119 = f'{ma.loc["2014-12-01", "Total_19-21"]:.1f}'
t1122 = f'{ma.loc["2014-12-01", "Total_22-24"]:.1f}'
t1919 = f'{ma.loc["2020-02-01", "Total_19-21"]:.1f}'
t1922 = f'{ma.loc["2020-02-01", "Total_22-24"]:.1f}'
tlt19 = f'{ma.loc[cps_date(), "Total_19-21"]:.1f}'
tlt22 = f'{ma.loc[cps_date(), "Total_22-24"]:.1f}'
ltdate = dtxt(cps_date())['mon1']    
text = ('From 1994 to 2000, labor force participation among young people '+
        'increased slightly. Following the recession of 2001, and carrying '+
        'through the great recession, participation rates dropped sharply. '+
        'From 2000 to 2014, annual labor force non-participation increased '+
        f'from {t0019} percent to {t1119} percent for 19 to 21 year olds '+
        f'and from {t0022} percent to {t1122} percent for 22 to 24 year olds '+
        '(see {\color{orange!90!yellow}\\textbf{---}}). The overall '+
        'increase is nearly entirely accounted for by increased college '+
        'enrollment (see {\color{blue!50!cyan}\\textbf{---}}).\n\nBy February '+
        '2020, the labor market had improved and the annual non-participation '+
        f'rate was {t1919} percent for 19 to 21 year olds and {t1922} '+
        f'percent for 22 to 24 year olds. In the latest data, covering the '+
        f'12 months ending {ltdate}, the rate of non-participation is {tlt19} '+
        f'percent for 19 to 21 year olds and {tlt22} percent for 22 to 24 '+
        'year olds. ')
write_txt(text_dir / 'young_nilf.txt', text)
print(text)

### CPS Labor Force Nonparticipation by Reason

Replication using python of a very clever chart by Ernie Tedeschi (@ernietedeschi).

[Definitions](https://www.frbatlanta.org/chcs/human-capital-currents/2015/0612-measuring-labor-market-status-using-basic-data.aspx) of labor market status come from the FRB of Atlanta.

In [ ]:
years = range(2000, 2023)
ref_pd = (2001, 3)
ref_dt = to_date(ref_pd)
ages = '18 <= AGE <= 64'
wgt = 'BASICWGT'
cat = 'NILFREASON'
dates = ['YEAR', 'MONTH']
dems = ['FEMALE', 'AGE']
cols = dates + dems + [wgt, cat]
files = [f'cps{year}.ft' for year in years]

# CPS Data
df = pd.concat([pd.read_feather(cps_dir / file, columns=cols)
                  .query(ages) for file in files])

p = df.groupby(dates + dems)[wgt].sum().rename('ADJ')
sh = (p / p.groupby(dates).sum())
adj = (sh.loc[ref_pd] / sh).reset_index()
data = (pd.merge(df, adj)
          .assign(ADJWGT = lambda x: x.ADJ * x[wgt]))
data.NILFREASON.cat.rename_categories({'nan': 'LF'}, 
                                      inplace=True)

# Make Adjustments
c = data.groupby(dates + [cat]).ADJWGT.sum()
dem_res = (c / c.groupby(dates).sum()).unstack()
dem_res.columns = dem_res.columns.to_list()

c_nd = data.groupby(dates + [cat])[wgt].sum()
nd_res = (c_nd / c_nd.groupby(dates).sum()).unstack()

dem_res['Demographics'] = dem_res['LF'] - nd_res['LF']
dem_res.index = [to_date(ym) for ym in dem_res.index]

keep_cols = dem_res.columns.difference(['LF'])
result = (dem_res.rolling(12).mean().dropna()
                 .loc[ref_dt:, keep_cols] * 100)

final = result.iloc[0] - result
final.to_csv(data_dir / 'nilf.csv', index_label='date')

In [ ]:
final = pd.read_csv(data_dir / 'nilf.csv', index_col='date', 
                    parse_dates=True)
colors = {'Disabled/Ill': 'green!80!blue', 
          'Discouraged': 'blue', 
          'Family': 'red', 
          'Retired': 'orange', 
          'School': 'cyan', 
          'Demographics': 'violet!80!purple'}

adj = node_adj(final)
node_file = open(text_dir / 'nilf_nodes.txt', 'w')
for series, color in colors.items():
    if series in adj.keys():
        offset = adj[series] / 2
    else:
        offset = 0
    node_file.write(end_node(final[series], color, 
                             size=1.2, offset=offset))
node_file.close()

ltdate = dtxt(cps_date())['mon1']
lt = final.iloc[-1]
tot = abs(lt.sum())
dem = abs(lt['Demographics'])
demc = c_line(colors['Demographics'])
sch = abs(lt['School'])
schc = c_line(colors['School'])
dis = abs(lt['Disabled/Ill'])
disc = c_line(colors['Disabled/Ill'])
ret = abs(lt['Retired'])
retc = c_line(colors['Retired'])

text = (f'From March 2001 to the latest available twelve months '+
        f'of data, ending {ltdate}, an additional {tot:.1f} percent '+
         'of the age 18--64 population left the labor force. Changes '+
         'in the demographic composition of the population affect the '+
         'rate of participation. For example, the larger-than-normal '+
         'population cohort born after World War II is reaching '+
         'retirement age in this period. Changes in the age and '+
        f'sex distribution within the age group explain {dem:.1f} '+
        f'percentage points of the cumulative decrease in participation '+ 
        f'since March 2001 {demc}. \n\nAdditionally, young people are '+
        f'staying in school longer, on average, reducing the age 18--64 '+
        f'labor force by {sch:.1f} percent {schc}. Disability and '+
        f'illness reduce the labor force by another {dis:.1f} percent '+
        f'{disc}. Less retirement among those age 18--64 increases the '+
        f'labor force by {ret:.1f} percent, over the period {retc}.')
write_txt(text_dir / 'nilf_01.txt', text)
print(text)

In [ ]:
# Bar chart with NILF categories latest and pre-COVID
result, share = pd.DataFrame(), pd.DataFrame()
dts = [cps_date(), cps_date() - pd.DateOffset(years=2)]
for i, date in enumerate(dts):
    m = cps_1mo(cps_dir, date, cols + ['PWSSWGT']).query('AGE > 15')
    dt = dtxt(date)['mon1']
    result[dt] = m.groupby('NILFREASON').PWSSWGT.sum()
    share[dt] = result[dt] / result[dt].sum() * 100
    write_txt(text_dir / f'nilf_mo{i+1}.txt', dtxt(date)['mon2'])
    
(share.drop('nan').sort_values(share.columns[0], ascending=False)
 .round(1).to_csv(data_dir / 'nilf_comp.csv', index_label='name'))

sh = share.applymap('{:.1f} percent'.format)
res = result.applymap(lambda x: f'{x / 1_000_000:.1f} million')
mo1, mo2 = sh.columns
totlt = result[mo1].drop('nan').sum() / 1_000_000
totsh = share[mo1].drop('nan').sum()
shpr = share[mo2].drop('nan').sum()

color = 'green!80!blue!72!black'
color2 = 'green!80!blue!60!lime'

ret = 'Retired'
di = 'Disabled/Ill'
fm = 'Family'
sc = 'School'

text = (f'Nonparticipants aged 16 and older total {totlt:.1f} '+
        f'million in {mo1}, and make up {totsh:.1f} percent of the '+
        f'age 16 or older population, compared to {shpr:.1f} '+
        f'percent in {mo2}. About half of nonparticipants, and '+
        f'{sh.loc[ret, mo1]} of the population, are '+
        f'retirees in {mo1} {c_box(color)}, compared to '+
        f'{sh.loc[ret, mo2]} percent in '+
        f'{mo2} {c_box(color2)}.\n\nDisability or illness keeps an '+
        f'additional {sh.loc[di, mo1]} '+
        f'out of the labor force in {mo1}, compared to '+
        f'{sh.loc[di, mo2]} in {mo2}. Students who are out of the '+
        f'labor force make up {sh.loc[sc, mo1]} '+
        f'in {mo1} and {sh.loc[sc, mo2]} in {mo2}, '+
        f'while unpaid caregivers are {sh.loc[fm, mo1]} in {mo1} and '+
        f'{sh.loc[fm, mo2]} in {mo2}.')
write_txt(text_dir / 'nilfbasic.txt', text)
print(text)

### Occupations

In [ ]:
occs = {1   :  'Management occupations',
        2   :  'Business and financial operations occupations',
        3   :  'Computer and mathematical science occupations',
        4   :  'Architecture and engineering occupations',
        5   :  'Life, physical, and social science occupations',
        6   :  'Community and social service occupations',
        7   :  'Legal occupations',
        8   :  'Education, training, and library occupations',
        9   :  'Arts, design, entertainment, sports, and media',
        10  :  'Healthcare practitioner and technical occupations',
        11  :  'Healthcare support occupations',
        12  :  'Protective service occupations',
        13  :  'Food preparation and serving related occupations',
        14  :  'Building and grounds cleaning and maintenance',
        15  :  'Personal care and service occupations',
        16  :  'Sales and related occupations',
        17  :  'Office and administrative support occupations',
        18  :  'Farming, fishing, and forestry occupations',
        19  :  'Construction and extraction occupations',
        20  :  'Installation, maintenance, and repair occupations',
        21  :  'Production occupations',
        22  :  'Transportation and material moving occupations'}

cols = ['MONTH', 'YEAR', 'LFS', 'PWSSWGT', 'OCC03D', 'OCC203D', 'WORKFT', 'COW1', 'COW2']

df = cps_3mo(cps_dir, cps_date(), cols)
df2 = cps_3mo(cps_dir, cps_date() - pd.DateOffset(months=24), cols)

df['OCC03D'] = df['OCC03D'].map(occs)
df2['OCC03D'] = df2['OCC03D'].map(occs)

data = pd.DataFrame()

data['FT'] = (((df.query('WORKFT == 1').groupby('OCC03D').PWSSWGT.sum() / df.PWSSWGT.sum()) - 
              (df2.query('WORKFT == 1').groupby('OCC03D').PWSSWGT.sum() / df.PWSSWGT.sum())) * 100)
data['PT'] = (((df.query('WORKFT == 0').groupby('OCC03D').PWSSWGT.sum() / df.PWSSWGT.sum()) - 
              (df2.query('WORKFT == 0').groupby('OCC03D').PWSSWGT.sum() / df.PWSSWGT.sum())) * 100)
data['Total'] = data.sum(axis=1)

data.index = data.index.str.replace('occupations', '').str.replace('and', '\&')

data = data.sort_values('Total')
data.drop('Total', axis=1).to_csv(data_dir / 'occs.csv', sep=';', index_label='name', header=True)

data.drop('Total', axis=1).plot(kind='barh', stacked=True);

data1 = data.drop('Total', axis=1)

### Atlanta Fed WGT

In [ ]:
cols = ['AGE', 'HRWAGE', 'ATLFLG', 'FEMALE', 'CPSID', 'WBHAO', 
        'LINENO', 'WKEARN']
mergecol = ['CPSID', 'LINENO', 'FEMALE', 'WBHAO']
filters = 'HRWAGE >= 2.13 and ATLFLG == 1'
dates = pd.date_range('1997-01-01', cps_date(), freq='MS')
d, d2, d3 = {}, {}, {}
for dt in dates:
    df1 = cps_1mo(cps_dir, dt - pd.DateOffset(years=1), 
                  cols).query(filters)
    df2 = cps_1mo(cps_dir, dt, cols).query(filters)
    data = (pd.merge(df1, df2, on=mergecol))
    data = (data.loc[(data.AGE_x <= data.AGE_y) & 
                     (data.AGE_y - 2 <= data.AGE_x)])
    wage_change_array = ((data['HRWAGE_y'] / data['HRWAGE_x']) - 1) * 100
    wkpy_change_array = ((data['WKEARN_y'] / data['WKEARN_x']) - 1) * 100
    d[dt] = wage_change_array.median()
    d3[dt] = wkpy_change_array.median()
    zwc = (len(wage_change_array[(wage_change_array >= -0.5) & 
                                 (wage_change_array <= 0.5)]) / 
           len(wage_change_array)) * 100
    d2[dt] = zwc

result = pd.Series(d, name='bd_cps').to_frame()
result['3ma'] = result.rolling(3).mean()
result['wk'] = pd.Series(d3, name='wkpy').to_frame()
result['wk3ma'] = result['wk'].rolling(3).mean()
result['zwc'] = pd.Series(d2).rolling(3).mean()
result.to_csv(data_dir/ 'atl_wgt.csv', index_label='date')

In [ ]:
df = pd.read_csv(data_dir / 'atl_wgt.csv', 
                 index_col='date', parse_dates=True)
ltdt = dtxt(df.index[-1])['mon1']
ltval = df['bd_cps'].iloc[-1]
lt3m = df['3ma'].iloc[-1]
yrdt = dtxt(df.index[-13])['mon1']
pr3m = df['3ma'].iloc[-13]
also = 'also' if round(lt3m) == round(pr3m) else ''

col = 'orange!60!white'
col3 = 'blue!75!cyan'
wval = value_text(df.wk.iloc[-1])
text = ('Replication of the wage growth tracker '+
        'shows matched-observation hourly wage '+
        f'growth of {ltval:.1f} percent in {ltdt} {c_line(col)}, '+
        f'and average wage growth of {lt3m:.1f} percent over '+
        f'the three months ending {ltdt} {c_line(col3)}. '+
        f'One year prior, in {yrdt}, three-month moving average '+
        f'wage growth was {also}{pr3m:.1f} percent. Matched '+
        'observation weekly wage growth, which is affected '+
        f'by changes in hours worked, {wval} over the year '+
        f'ending {ltdt} (see right chart).')
write_txt(text_dir / 'atl_wgt.txt', text)
print(text)

ltval = df['zwc'].iloc[-1]
prval = df['zwc'].iloc[-2]
prdate = dtxt(df.index[-2])['mon1']
yrval = df['zwc'].iloc[-13]

zcol = 'red'
node = end_node(df['zwc'], zcol, date='m', 
                full_year=True, percent=True)
write_txt(text_dir/ 'zwc_node.txt', node)

text = (f'In {ltdt}, {ltval:.1f} '+
        'percent of individuals had no hourly wage growth, '+
        f'compared to {prval:.1f} in {prdate} {c_line(zcol)}. '+
        f'One year prior, in {yrdt}, {yrval:.1f} percent of '+
        'individuals had no wage growth.')
write_txt(text_dir / 'atl_zwc.txt', text)
print(text)

### Unemployment Rate for Smaller Groups

In [ ]:
cols = ['YEAR', 'MONTH', 'WBHAOM', 'BASICWGT', 'LFS', 'AGE', 'FEMALE']
groups = ['White', 'Black', 'Hispanic', 'Asian', 'Native American', 'More than one']

unrate = lambda x: np.average(x['UNEMP'], weights=x['BASICWGT']) * 100
unsh = lambda x: (x['UNEMP'] * x['BASICWGT']).sum()
unemp = lambda x: np.where(x['LFS'] == 'Unemployed', 1, 
                  np.where(x['LFS'] == 'Employed', 0, np.nan))


df, df2, df3 = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
for dt in [cps_date(), cps_date() - pd.DateOffset(months=24)]:
    data = (cps_6mo(cps_dt=pd.to_datetime(dt), cols=cols, cps_dir=cps_dir)
            .assign(UNEMP = unemp)
            .query('AGE > 15 and AGE < 65 and LFS != "NILF"'))
    
    srs = data.groupby('WBHAOM').apply(unrate)
    df[dtxt(dt)['mon2']] = srs.loc[groups]
    
    unemptot = (data['UNEMP'] * data['BASICWGT']).sum()
    srs2 = (data.groupby('WBHAOM').apply(unsh) / unemptot) * 100
    df2[dtxt(dt)['mon2']] = srs2.loc[groups]
    pop = data.BASICWGT.sum()
    popsh = (data.groupby('WBHAOM').BASICWGT.sum() / pop) * 100
    df3[dtxt(dt)['mon2']] = popsh.loc[groups]

df.index.name = ''
df.to_csv(data_dir / 'unemp_grp.csv', index_label='name', header=True)
df2.index.name = ''
df2['pop'] = df3.mean(axis=1)
df2.to_csv(data_dir / 'unemp_grpsh.csv', index_label='name', header=True)

In [ ]:
cols = ['MONTH', 'YEAR', 'LFS', 'PWSSWGT', 'AGE', 'WBHAOM', 'FEMALE']
df1 = cps_6mo(cps_dir, cps_date() - pd.DateOffset(months=24), cols)
df2 = cps_6mo(cps_dir, cps_date(), cols)

ages = [(18, 64), (25, 54), (16, 24), (25, 34), (35, 44), (45, 54), (55, 64), (65, 74)]
groups = ['White', 'Black', 'Hispanic', 'Asian', 'Native American', 'More than one']
sex = [[0], [1], [0, 1]] 

results = pd.DataFrame()
for a, g, s in itertools.product(ages, groups, sex):
    row_name = f'{a[0]}--{a[1]}'
    col_name = f'{g} {["Female" if s == [1] else "Male" if s == [0] else "Total"][0]}'
    for yr, df in [('Year Ago', df1), ('Latest', df2)]:
        data = df.query('@a[0] <= AGE <= @a[1] and WBHAOM == @g and FEMALE in @s')
        i = data.groupby('LFS').PWSSWGT.sum()
        results.loc[row_name, f'{yr} {col_name}'] = (i.Unemployed / (i.Unemployed + i.Employed)) * 100
        
results.iloc[:, :-12].to_csv(data_dir / 'unempgroups.csv', index_label='name')
results.iloc[:2, -12:].to_csv(data_dir / 'unempgroups2.csv', index_label='name')

In [ ]:
ages = [(18, 64), (25, 54), (16, 24), (25, 34), (35, 44), (45, 54), (55, 64), (65, 74)]
sex = [[0], [1], [0, 1]] 

results = pd.DataFrame()
for a, s in itertools.product(ages, sex):
    row_name = f'{a[0]}--{a[1]}'
    col_name = f'{["Female" if s == [1] else "Male" if s == [0] else "Total"][0]}'
    for yr, df in [('Year Ago', df1), ('Latest', df2)]:
        data = df.query('@a[0] <= AGE <= @a[1] and FEMALE in @s')
        i = data.groupby('LFS').PWSSWGT.sum()
        results.loc[row_name, f'{yr} {col_name}'] = (i.Unemployed / (i.Unemployed + i.Employed)) * 100
        
results.to_csv(data_dir / 'unempgroups3.csv', index_label='name')

### Usual hours worked

In [ ]:
hrs = lambda x: np.where(x.HRSUSL1 == -4, x.HRSUSL1I,
                np.where(x.HRSUSL1 > 0, x.HRSUSL1,
                         np.nan))

wgt_avg = lambda x: np.average(x.HRS, weights=x.BASICWGT)

cols = ['HRSUSL1', 'HRSUSL1I', 'BASICWGT', 'LFS', 'YEAR', 'MONTH']

data = pd.concat([(pd.read_feather(f'{cps_dir}/cps{year}.ft', columns=cols)
                     .assign(HRS = hrs).query('HRS > 0'))
                  for year in range(2018, 2023)])

grp_data = data.groupby(['YEAR', 'MONTH']).apply(wgt_avg)
grp_data.index = [to_date(ym) for ym in grp_data.index]

sa_cps = x13_arima_analysis(grp_data).seasadj

sa_cps.to_csv(data_dir / 'uslhrs.csv', index_label='name')

### Labor Force Participation

In [ ]:
final = pd.DataFrame()
years = range(1989, 2023)
compdt = (2000, 1)
dates = ['YEAR', 'MONTH']
dems = ['AGE', 'FEMALE']
cols = dates + ['AGE', 'FEMALE', 'BASICWGT', 'LFS']
files = [f'cps{year}.ft' for year in years]
sex = [('Men', [0]), ('Women', [1]), ('Total', [0, 1])]
for name, grp in sex:
    query = 'AGE >= 16 and FEMALE in @grp'
    # CPS Data
    df = pd.concat([pd.read_feather(cps_dir / file, columns=cols)
                      .query(query) for file in files])
    p = df.groupby(dates + dems)[wgt].sum().rename('ADJ')
    sh = (p / p.groupby(dates).sum())
    adj = (sh.loc[compdt] / sh).reset_index()
    data = (pd.merge(df, adj)
              .assign(ADJWGT = lambda x: x.ADJ * x[wgt]))
    wgts = ['BASICWGT', 'ADJWGT']
    res = pd.DataFrame({wgt: (1 - (data.groupby(dates + ['LFS'])[wgt].sum() / 
                                   data.groupby(dates)[wgt].sum())
                              .unstack()['NILF']
                             ).multiply(100).dropna()
                  for wgt in wgts})
    res.index = [to_date(ym) for ym in res.index]
    sa = pd.DataFrame({f'{i}_{name}': x13_arima_analysis(res[i]).seasadj 
                       for i in wgts})    
    final = pd.concat([final, sa], axis=1)
final.to_csv(data_dir / 'lfpr_adj.csv', index_label='date')

In [ ]:
df = pd.read_csv(data_dir / 'lfpr_adj.csv', index_col='date', 
                 parse_dates=True)
ltdt = dtxt(df.index[-1])['mon1']
write_txt(text_dir / 'lfpr_cpsdate.txt', ltdt)
colors = {'Total': 'green!70!lime', 'Men': 'blue!40!cyan', 
          'Women': 'orange!40!yellow'}
nodes = (end_node(df['ADJWGT_Men'], colors['Men'], 
                  percent=True, date='m', full_year=True, 
                  offset=0.35) + '\n' + 
         '\n'.join(end_node(df[f'ADJWGT_{name}'], color, 
                            percent=True) 
                   for name, color in colors.items() 
                   if name != 'Men'))
write_txt(text_dir / 'lfpr_nodes_adj.txt', nodes)

lt = final.iloc[-1]
diffval = lt['ADJWGT_Total'] - lt['BASICWGT_Total']
text = ('Reweighting the population to match the '+
        f'age composition in {compdt[0]} suggests the aging '+
        f'of the US population since {compdt[0]} has reduced '+
        f'total labor force participation by {diffval:.1f} '+
        f'percentage points. ')
write_txt(text_dir / 'lfpr_adj.txt', text)
print(text)

### Self Employment pre-2000 data

In [ ]:
cols = ['MONTH', 'YEAR', 'AGE', 'LFS', 'COW1', 'BASICWGT']
data = pd.concat([(pd.read_feather(f'{cps_dir}/cps{year}.ft', columns=cols)
                     .query('LFS in ["Employed", "Unemployed"]'))
                  for year in range(1989, 2001)])
inc = lambda x: np.where((x.COW1 == "Self-employed Incorporated") 
                         & (x.LFS == "Employed"), x.BASICWGT, 0)
data = data.assign(INC = inc)
grp_data = data.groupby(['YEAR', 'MONTH']).INC.sum()
grp_data.index = [to_date(ym) for ym in grp_data.index]
grp_data.to_csv(data_dir / 'se_inc_hist.csv', index_label='date', 
                header=True)

### Wages and Education

In [11]:
cols = ['MONTH', 'YEAR', 'AGE', 'EDUC', 'PWORWGT', 'WKEARN', 'WORKFT', 
        'PRICEADJ']
edgrp = lambda x: np.where((x.EDUC.isin(['LTHS', 'HS'])), 'LTHS/HS',
                  np.where((x.EDUC == 'SC'), 'SC', 'COLL/ADV'))
rw = lambda x: x.WKEARN * x.PRICEADJ
df = pd.concat([(pd.read_feather(f'{cps_dir}/cps{year}.ft', columns=cols)
                     .query('WKEARN > 0 and WORKFT == 1 and 25 <= AGE <= 54')
                     .assign(EDGRP = edgrp, RW = rw))
                  for year in range(1989, 2023)])

ed = df.groupby(['YEAR', 'MONTH', 'EDGRP']).PWORWGT.sum().unstack()
ed.index = [pd.to_datetime(f'{i[0]}-{i[1]}-01') for i in ed.index]
ed = ed.divide(ed.sum(axis=1), axis=0)
df = df.merge((ed.loc['2019'].mean() / ed.iloc[-1]).reset_index())
df['ADJWGT'] = df['PWORWGT'] * df[0]

data = (df.groupby(['YEAR', 'MONTH', 'EDGRP'])
          .apply(binned_wage, wage_var='RW', percentile=0.5).unstack())
data.index = [pd.to_datetime(f'{i[0]}-{i[1]}-01') for i in data.index]
data2 = (df.groupby(['YEAR', 'MONTH'])
           .apply(binned_wage, wage_var='RW', percentile=0.5))
data2.index = [pd.to_datetime(f'{i[0]}-{i[1]}-01') for i in data2.index]
data['Total'] = data2

In [129]:
cdt = pd.read_csv(data_dir / 'cpi.csv', index_col='date', parse_dates=True).index[-1]
data = data.loc[:cdt]
cq = f'MONTH == {cdt.month} and YEAR == {cdt.year}'
rwe = binned_wage(df.query(cq), wage_var='RW', percentile=0.5, wgt_var='ADJWGT')
val19 = data.loc['2019'].mean()
vallt = data.iloc[-1]
res = (((vallt / val19) - 1) * 100).rename('Total').to_frame()
res['Composition'] = 0
rev = (((rwe / val19.Total) - 1) * 100)
res.loc['Total', 'Composition'] = res.loc['Total', 'Total'] - rev
res['Value'] = res['Total'] - res['Composition']
names = {'COLL/ADV': "Bachelor's degree or more",
         'LTHS/HS': 'High school degree or less',
         'SC': 'Some college or associate degree',
         'Total': 'Any education level'}
res.rename(names).to_csv(data_dir / 'educ_wage_bar.csv', index_label='group')
ltdt = dtxt(data.index[-1])['mon1']
write_txt(text_dir / 'rw_cpsdate.txt', ltdt)

d3m = data.rolling(3).mean().dropna()
d3m.to_csv(data_dir / 'educ_wage.csv', index_label='date')
adj = node_adj(d3m)
smax = d3m.iloc[-1].idxmax()
adj[smax] = adj[smax] + 0.35

cols = {'COLL/ADV': 'green!80!blue', 
        'LTHS/HS': 'blue!90!cyan',
        'SC': 'cyan',
        'Total': 'orange!70!yellow'}
date = {series: 'm' if series == smax else None 
        for series in cols.keys()}
nodes  ='\n'.join([end_node(d3m[series], color, 
                            date=date[series], 
                            full_year=True, dollar=True,
                            digits='comma',
                            size=1.1, offset=adj[series]) 
                   for series, color in cols.items()])
write_txt(text_dir / 'educ_wage_nodes.txt', nodes) 
ch = ((d3m / d3m.iloc[0]) - 1) * 100
ltdt = dtxt(cdt)['mon1']
chlt = ch.iloc[-1]
totlt = d3m.Total.iloc[-1]

totch = value_text(chlt.Total, 'increase_by')
bach = value_text(chlt['COLL/ADV'])
scch = value_text(chlt['SC'])
hsch = value_text(chlt['LTHS/HS'])

text = (f'Over the three months ending {ltdt}, the median '+
        'usual earnings of full-time wage and salary workers '+
        f'age 25--54 averaged \${totlt:,.0f} per week. After '+
        f'adjusting for inflation, these earnings have {totch}, '+
        'in total, since 1989. Digging deeper, the workforce is split '+
        'into three groups by highest level of education attained. '+
        f'Real earnings {bach} over the same period for workers '+
        f"with bachelor's degree or more, {scch} for workers with "+
        f'some college or an associate degree, and {hsch} for '+
        'those with a high school degree or less.')
write_txt(text_dir / 'educ_wage1.txt', text)
print(text)

size = ('the majority' 
        if res.loc['Total', 'Composition'] > res.loc['Total', 'Value'] 
        else 'much')
even = ('While real wage growth since 2019 has been uneven' 
        if (res.drop('Total').Total.max() - res.drop('Total').Total.min()) > 2
        else ('While wage growth since 2019 has been fairly even '+
        'among education groups,'))
text = ('Real wage growth is tied to changes in education, even in '+
        f'the short-term. {even}, {size} of the overall '+
        'increase can be explained by increases in the education '+
        'level of the workforce.')
write_txt(text_dir / 'educ_wage2.txt', text)
print(text)

names2 = {'COLL/ADV': "a bachelor's degree or more",
          'LTHS/HS': 'a high school degree or less',
          'SC': 'some college and no degree or an associate degree'}
tot = res.drop('Total').rename(names2).Total.sort_values()
cat1 = tot.index[2]
cat1ch = value_text(tot.iloc[2])
cat2 = tot.index[1]
cat2ch = value_text(tot.iloc[1])
cat3 = tot.index[0]
cat3ch = value_text(tot.iloc[0])
tot = res.loc['Total', 'Total']
comp = res.loc['Total', 'Composition']
text = (f'Since 2019, real wage growth was strongest for workers with {cat1}. '+
        'The real wages of full-time wage and salary workers age 25--54 '+
        f'with {cat1} {cat1ch}, in total, from 2019 to '+
        f'{ltdt} {c_box("green!70!blue")}. Real wages of the equivalent '+
        f'group with {cat2} {cat2ch}. Real wages {cat3ch} for those '+
        f'with {cat3}. '+
        '\n\nCombining the education groups, total real wage growth for '+
        'full-time workers age 25--54 with any education level was '+
        f'{tot:.1f} percent. Of this, {comp:.1f} percentage points are '+
        'explained by increases in the \\textbf{education-level '+
        'composition} of the overall group. ')
write_txt(text_dir / 'educ_wage3.txt', text)
print(text)

Over the three months ending February 2022, the median usual earnings of full-time wage and salary workers age 25--54 averaged \$1,095 per week. After adjusting for inflation, these earnings have increased by 11.5 percent, in total, since 1989. Digging deeper, the workforce is split into three groups by highest level of education attained. Real earnings increased 8.6 percent over the same period for workers with bachelor's degree or more, decreased 8.2 percent for workers with some college or an associate degree, and decreased 0.6 percent for those with a high school degree or less.
Real wage growth is tied to changes in education, even in the short-term. While real wage growth since 2019 has been uneven, the majority of the overall increase can be explained by increases in the education level of the workforce.
Since 2019, real wage growth was strongest for workers with a high school degree or less. The real wages of full-time wage and salary workers age 25--54 with a high school degre